In [1]:
import time
import os

import copy
from torch.utils.data import DataLoader

import torch
import numpy as np
import random

from experiment.helpers import get_data_training, get_model, get_optimizer, get_scheduler, get_criterion, \
    store_training_history, store_trained_model, get_unlearning_algorithm, get_dummy_model
from utils.train_utils import base_fit

from utils.unlearning_utils import RandomDistributionGenerator

from utils.unlearning_metrics import js_ind, membership_inference, pretty_predict_all, sape


from argparse import Namespace

# Reproducibility

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Random seed set as {seed}")

# Training Parameters

In [3]:
# params
full_train_args = Namespace(
    dataset='mufac', # CIFAR dataset
    mode='full', # do not get the full training set
    model='resnet18', # the model to use
    lr=1e-3, # learning rate
    optimizer='adam', # optimizer
    decay=0, # weight decay
    epochs=30, # number of epochs
    batch_size=64, # batch size
    scheduler=True, # 
    class_weights=False,
    store_model=False,
    store_history=False,
    num_runs=1
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Traditional Training

In [4]:
def train(args):
    set_seed(0)
    train_loader, val_loader, test_loader, forget_loader = get_data_training(args)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for i in range(args.num_runs):
        model = get_model(args)
        model.to(device)

        optimizer = get_optimizer(args, model)
        scheduler = get_scheduler(args, optimizer)

        criterion = get_criterion(args, train_loader, device)
        model, history = base_fit(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            forget_loader=None,
            optimizer=optimizer,
            scheduler=scheduler,
            criterion=criterion,
            epochs=args.epochs,
            return_history=True,
            device=device,
        )
        timestamp = str(time.time())
        name = str(args.model) + "_" + str(args.mode) + "_" + str(args.dataset) + "_" + timestamp
        if args.store_history:
            fig_name = name + "_history"
            store_training_history(args, history, fig_name)
        if args.store_model:
            store_trained_model(args, model, name)
        return model

# Train the full model

In [5]:
trained_model = train(full_train_args)

Random seed set as 0
#Training: 10025
#Validation: 1539
#Test: 1504
#Retain: 8525
#Forget: 1500
Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    RandomAffine(degrees=[0.0, 0.0], scale=(0.8, 1.2), shear=[-10.0, 10.0])
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=None)
    ToTensor()
)
Using seed=2357136044
Using seed=2546248239
Using seed=3071714933
Adjusting learning rate of group 0 to 1.0000e-03.


Epoch 1 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:09<00:00, 16.69it/s]

Adjusting learning rate of group 0 to 9.9726e-04.


[Epoch 1]
	[Train]	Loss=1.7425, Acc=0.2298
	[Val]	Loss=2.2437, Acc=0.1780
	[Test]	Loss=2.1579, Acc=0.2088


Epoch 2 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:08<00:00, 19.32it/s]

...

Adjusting learning rate of group 0 to 2.7391e-06.


[Epoch 29]
	[Train]	Loss=0.1315, Acc=0.9628
	[Val]	Loss=1.8771, Acc=0.5530
	[Test]	Loss=1.5374, Acc=0.5964


Epoch 30 - Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:08<00:00, 19.13it/s]

Adjusting learning rate of group 0 to 0.0000e+00.


[Epoch 30]
	[Train]	Loss=0.1296, Acc=0.9638
	[Val]	Loss=1.8497, Acc=0.5575
	[Test]	Loss=1.5223, Acc=0.6037


# Train the ideal model

In [6]:
retain_args = full_train_args
retain_args.mode = 'retain'
retain_model = train(retain_args)

Random seed set as 0
#Training: 10025
#Validation: 1539
#Test: 1504
#Retain: 8525
#Forget: 1500
Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    RandomAffine(degrees=[0.0, 0.0], scale=(0.8, 1.2), shear=[-10.0, 10.0])
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=None)
    ToTensor()
)
Using seed=2357136044
Using seed=2546248239
Using seed=3071714933
Adjusting learning rate of group 0 to 1.0000e-03.


Epoch 1 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:06<00:00, 19.26it/s]

Adjusting learning rate of group 0 to 9.9726e-04.


[Epoch 1]
	[Train]	Loss=1.6981, Acc=0.3223
	[Val]	Loss=1.5740, Acc=0.3788
	[Test]	Loss=1.4814, Acc=0.3989


Epoch 2 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:06<00:00, 19.55it/s]

...

Adjusting learning rate of group 0 to 2.7391e-06.


[Epoch 29]
	[Train]	Loss=0.1333, Acc=0.9646
	[Val]	Loss=1.9434, Acc=0.5348
	[Test]	Loss=1.6488, Acc=0.6011


Epoch 30 - Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:06<00:00, 19.49it/s]

Adjusting learning rate of group 0 to 0.0000e+00.


[Epoch 30]
	[Train]	Loss=0.1201, Acc=0.9653
	[Val]	Loss=1.9168, Acc=0.5250
	[Test]	Loss=1.6307, Acc=0.6004


In [7]:
def unlearning(model, args):
    args.unlearn = True
    if args.algorithm.lower() == "sftc":
        train_loader, val_loader, test_loader, forget_loader, merged_loader = get_data_training(args)
    else:
        train_loader, val_loader, test_loader, forget_loader = get_data_training(args)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for i in range(args.num_runs):
        model.to(device)

        optimizer = get_optimizer(args, model)
        scheduler = get_scheduler(args, optimizer)

        criterion = get_criterion(args, train_loader, device)

        unlearn_fn = get_unlearning_algorithm(args)

        if args.algorithm.lower() == "cfk":
            unlearn_args = {'eu_k': False, 'k': args.k}
        elif args.algorithm.lower() == "euk":
            unlearn_args = {'eu_k': True, 'k': args.k}
        elif args.algorithm.lower() == "neg_grad":
            unlearn_args = {'advanced_neg_grad': False}
        elif args.algorithm.lower() == "advanced_neg_grad":
            unlearn_args = {'advanced_neg_grad': True}
        elif args.algorithm.lower() == "scrub":
            teacher_model = copy.deepcopy(model)
            unlearn_args = {'teacher_model': teacher_model, 'temperature': args.temperature}
        elif args.algorithm.lower() == "bad_teaching":
            teacher_model = copy.deepcopy(model)
            dummy_model = get_model(args)
            dummy_model.to(device)
            unlearn_args = {'teacher_model': teacher_model, 'dummy_model': dummy_model, 'temperature': args.temperature}
        elif args.algorithm.lower() == "sftc":
            teacher_model = copy.deepcopy(model)
            dummy_model = get_dummy_model(args)
            if not isinstance(dummy_model, RandomDistributionGenerator):
                dummy_model.to(device)
            unlearn_args = {'teacher_model': teacher_model, 'dummy_model': dummy_model, 'temperature': args.temperature,
                            'confuse_fraction': args.confuse_fraction, 'merged_loader': merged_loader}
        elif args.algorithm.lower() == "sftc+":
            teacher_model = copy.deepcopy(model)
            dummy_model = get_dummy_model(args)
            if not isinstance(dummy_model, RandomDistributionGenerator):
                dummy_model.to(device)
            unlearn_args = {'teacher_model': teacher_model, 'dummy_model': dummy_model, 'temperature': args.temperature,
                            'confuse_fraction': args.confuse_fraction}
        else:
            unlearn_args = {}

        model, history = unlearn_fn(
            model=model,
            retain_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            forget_loader=forget_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            criterion=criterion,
            epochs=args.epochs,
            return_history=True,
            device=device,
            **unlearn_args
        )
        timestamp = str(time.time())
        name = str(args.model) + "_" + str(args.algorithm) + "_" + str(args.dataset) + "_" + timestamp
        if args.store_history:
            fig_name = name + "history"
            store_training_history(args, history, fig_name)
        if args.store_model:
            store_trained_model(args, model, name, unlearn=True)
        return model

In [8]:
def evaluate(pretrained_model, retain_model, unlearned_model, unlearn_args):
    dataset = unlearn_args.dataset
    retain_set, val_set, test_set, forget_set = get_data_training(unlearn_args, return_sets=True)
    
    retain_dataloader = DataLoader(retain_set, batch_size=unlearn_args.batch_size, shuffle=False)
    test_dataloader = DataLoader(test_set, batch_size=unlearn_args.batch_size, shuffle=False)
    forget_dataloader = DataLoader(forget_set, batch_size=unlearn_args.batch_size, shuffle=False)

    set_seed(0)
    js_div = js_ind(
        unlearned_model=unlearned_model, original_model=pretrained_model,
        forget_loader=forget_dataloader,
        device=device
    )
    print("JS distinguishability", js_div)
    print("\n\n")
    
        

    mia_cv, mia_forget = membership_inference(
        model=unlearned_model, retain_set=retain_set, test_set=test_set, forget_set=forget_set, 
        verbose=False, full_verbose=False, device=device
    )
    print('[Unlearned Model]')
    print("MIA CV", mia_cv)
    print("MIA Forget", mia_forget)
    print("\n\n")


    report = pretty_predict_all(
        [unlearned_model, pretrained_model, retain_model],
        ['Unlearned', 'Original', 'Retain'],
        {
            'forget': forget_dataloader, 
            'test': test_dataloader, 
        },
        device
    )
    print(report)
    print("\n")
    unlearn_acc_forget = report[(report.Model == 'Original') & (report.Dataset == 'forget')]['Acc'].values[0]
    retain_acc_forget = report[(report.Model == 'Unlearned') & (report.Dataset == 'forget')]['Acc'].values[0]
    unlearn_acc_test = report[(report.Model == 'Unlearned') & (report.Dataset == 'test')]['Acc'].values[0]
    retain_acc_test = report[(report.Model == 'Unlearned') & (report.Dataset == 'test')]['Acc'].values[0]
    original_acc_test = report[(report.Model == 'Original') & (report.Dataset == 'test')]['Acc'].values[0]
                
    sape_eff_r = sape(retain_acc_test, unlearn_acc_test)
    sape_dis_r = sape(retain_acc_forget, unlearn_acc_forget)
    sape_eff = sape(original_acc_test, unlearn_acc_test)
                
     

In [9]:
# params
unlearn_args = Namespace(
    dataset='mufac', # CIFAR dataset
    algorithm='sftc', # unlearning alg
    mode='retain',
    model='resnet18', # the model to use
    dummy_model='random',
    lr=9e-4, # learning rate
    optimizer='adam', # optimizer
    decay=0, # weight decay
    epochs=4, # number of epochs
    batch_size=64, # batch size
    scheduler=False, # 
    class_weights=False,
    store_model=False,
    store_history=False,
    num_runs=1,
    temperature=1.,
    confuse_fraction=0.
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Perform Unlearning

In [10]:
tmp_model = copy.deepcopy(trained_model)
unlearned_model = unlearning(tmp_model, unlearn_args)

#Training: 10025
#Validation: 1539
#Test: 1504
#Retain: 8525
#Forget: 1500
Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)
Using seed=3626093760
Using seed=2588848963
Using seed=3684848379
Using seed=2340255427


Epoch 1 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:11<00:00, 14.00it/s]


[Epoch 1]
	[Train]	Retain Loss=0.3589, Retain Acc=0.9249, KL Loss=0.5304
	[Val]	Loss=1.4318, Acc=0.5062
	[Test]	Loss=1.2890, Acc=0.5645
	[Forget] Loss=0.8061, Acc=0.6953


Epoch 2 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:10<00:00, 14.28it/s]


[Epoch 2]
	[Train]	Retain Loss=0.1955, Retain Acc=0.9533, KL Loss=0.3107
	[Val]	Loss=1.6390, Acc=0.4893
	[Test]	Loss=1.3774, Acc=0.5572
	[Forget] Loss=1.0000, Acc=0.6367


Epoch 3 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:11<00:00, 13.97it/s]


[Epoch 3]
	[Train]	Retain Loss=0.1580, Retain Acc=0.9483, KL Loss=0.2692
	[Val]	Loss=1.5883, Acc=0.4919
	[Test]	Loss=1.4946, Acc=0.5399
	[Forget] Loss=1.1833, Acc=0.5793


Epoch 4 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:11<00:00, 13.75it/s]


[Epoch 4]
	[Train]	Retain Loss=0.1109, Retain Acc=0.9862, KL Loss=0.2172
	[Val]	Loss=1.5847, Acc=0.5055
	[Test]	Loss=1.4867, Acc=0.5652
	[Forget] Loss=1.1461, Acc=0.6073


In [11]:
unlearn_args

Namespace(dataset='mufac', algorithm='sftc', mode='retain', model='resnet18', dummy_model='random', lr=0.0009, optimizer='adam', decay=0, epochs=4, batch_size=64, scheduler=False, class_weights=False, store_model=False, store_history=False, num_runs=1, temperature=1.0, confuse_fraction=0.0, unlearn=True)

# Evaluate Unlearning

In [12]:
evaluate(trained_model, retain_model, unlearned_model, unlearn_args)

#Training: 10025
#Validation: 1539
#Test: 1504
#Retain: 8525
#Forget: 1500
Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)
Random seed set as 0
JS distinguishability 1.0537023544311523



[Unlearned Model]
MIA CV {'Accuracy': 0.8181563396553917, 'TP': 253.0, 'TN': 239.2, 'FP': 61.6, 'FN': 47.8, 'TPR': 0.8410897009966776, 'TNR': 0.7952314507198228}
MIA Forget {'TN': 1455.4, 'FP': 44.6, 'TNR': 0.9702666666666666}



       Model Dataset      Loss       Acc
0  Unlearned  forget  1.146621  0.607333
2   Original  forget  0.036783  0.994000
4     Retain  forget  2.349710  0.470667
1  Unlearned    test  1.486711  0.565160
3   Original    test  1.522299  0.603723
5     Retain    test  1.630721  0.600399


